In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [20]:
img_folder = r'D:\original'
out_folder = r'E:\ToAWS2'

course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [22]:

course_id = course_ids.get('중문')
course_id

'MGC003'

In [23]:
folders_candidate =glob.glob(os.path.join(out_folder,course_id,'*'))
folders_candidate

['E:\\ToAWS2\\MGC003\\20230910']

In [24]:
target_folder = folders_candidate[0]
target_folder

'E:\\ToAWS2\\MGC003\\20230910'

In [25]:
target_date = os.path.split(target_folder)[-1]
target_date 

'20230910'

Gather all data json files from Output Folder

In [26]:
dataJsonList = glob.glob(os.path.join(out_folder,course_id,target_date,'**/data*.json'), recursive=True)
dataJsonList

['E:\\ToAWS2\\MGC003\\20230910\\중문골프클럽 잔디보식지역\\data20230910중문골프클럽 잔디보식지역.json',
 'E:\\ToAWS2\\MGC003\\20230910\\한라 4H\\data20230910한라 4H.json',
 'E:\\ToAWS2\\MGC003\\20230910\\한라 5H\\data20230910한라 5H.json',
 'E:\\ToAWS2\\MGC003\\20230910\\한라 6H\\data20230910한라 6H.json',
 'E:\\ToAWS2\\MGC003\\20230910\\해안 1H\\data20230910해안 1H.json',
 'E:\\ToAWS2\\MGC003\\20230910\\해안 2H\\data20230910해안 2H.json',
 'E:\\ToAWS2\\MGC003\\20230910\\해안 6H\\data20230910해안 6H.json']

In [27]:
folderinDataJson = []
totalPhotoJson = []

for dataJson_ in dataJsonList:
  with open(dataJson_, "r", encoding='utf-8') as _json:
    dataJson= json.load(_json)


  for data_ in dataJson:
    # if data_['dest']['thumb'][0] !="M" :
      data_['dest']['thumb'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'thumb','thumbrgb{}.JPG'.format( data_['id']) )
    # if data_['dest']['rgb'][0] !="M" :
      data_['dest']['rgb'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'rgb','rgb{}.JPG'.format( data_['id']) )
    # if data_['dest']['ndvi'][0] !="M" :
      data_['dest']['ndvi'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'ndvi','ndvi{}.JPG'.format( data_['id']) )
    
  
  totalPhotoJson.extend(dataJson)

  for Json_ in dataJson:
    if Json_['destFolder'] not in folderinDataJson:
      folderinDataJson.append(Json_['destFolder'])




In [28]:
len(totalPhotoJson)

16961

In [29]:
totalPhotoJson[0]

{'id': '20230910131355_1264064138889_332512323889_92468000',
 'originalFileJPG': 'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/중문골프클럽 잔디보식지역/2023_0910_중문골프클럽_잔디보식지역_0.5m/20230910_1313-1345_잔디보식지역/DJI_20230910131355_0001_D.JPG',
 'originalFileGrp': ['ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/중문골프클럽 잔디보식지역/2023_0910_중문골프클럽_잔디보식지역_0.5m/20230910_1313-1345_잔디보식지역/DJI_20230910131355_0001_MS_G.TIF',
  'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/중문골프클럽 잔디보식지역/2023_0910_중문골프클럽_잔디보식지역_0.5m/20230910_1313-1345_잔디보식지역/DJI_20230910131355_0001_MS_NIR.TIF',
  'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/중문골프클럽 잔디보식지역/2023_0910_중문골프클럽_잔디보식지역_0.5m/20230910_1313-1345_잔디보식지역/DJI_20230910131355_0001_MS_R.TIF',
  'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/중문골프클럽 잔디보식지역/2023_0910_중문골프클럽_잔디보식지역_0.5m/20230910_1313-1345_잔디보식지역/DJI_20230910131355_0001_MS_RE.TIF'],
 'destFolder': '20230910/중문골프클럽 잔디보식지역/잔디보식지역_0',
 'GeoTagInfo': {'coords': [126.40641388888889,

In [30]:
save_name = os.path.join(out_folder,course_id,target_date, 'photo.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(totalPhotoJson, final , ensure_ascii=False)

In [31]:
filestructure = []


for folder_ in folderinDataJson:
  fileinfo = {}
  fileinfo["path"] = folder_
  fileinfo["file_cnt"] = len([ x for x in totalPhotoJson if x["destFolder"] == folder_])
  fileinfo["labeled_file_cnt"] = 0
  filestructure.append(fileinfo)
  

In [32]:
save_name = os.path.join(out_folder,course_id,target_date, 'filepath_'+course_id + '_'+target_date+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(filestructure, final , ensure_ascii=False)

In [33]:
filestructure

[{'path': '20230910/중문골프클럽 잔디보식지역/잔디보식지역_0',
  'file_cnt': 501,
  'labeled_file_cnt': 0},
 {'path': '20230910/중문골프클럽 잔디보식지역/잔디보식지역_1',
  'file_cnt': 501,
  'labeled_file_cnt': 0},
 {'path': '20230910/중문골프클럽 잔디보식지역/잔디보식지역_2',
  'file_cnt': 501,
  'labeled_file_cnt': 0},
 {'path': '20230910/중문골프클럽 잔디보식지역/잔디보식지역_3',
  'file_cnt': 501,
  'labeled_file_cnt': 0},
 {'path': '20230910/중문골프클럽 잔디보식지역/잔디보식지역_4',
  'file_cnt': 501,
  'labeled_file_cnt': 0},
 {'path': '20230910/중문골프클럽 잔디보식지역/잔디보식지역_5',
  'file_cnt': 501,
  'labeled_file_cnt': 0},
 {'path': '20230910/중문골프클럽 잔디보식지역/잔디보식지역_6',
  'file_cnt': 501,
  'labeled_file_cnt': 0},
 {'path': '20230910/중문골프클럽 잔디보식지역/잔디보식지역_7',
  'file_cnt': 4,
  'labeled_file_cnt': 0},
 {'path': '20230910/한라 4H/물리적피해,장비피해,잡초,광엽,피막이_0',
  'file_cnt': 501,
  'labeled_file_cnt': 0},
 {'path': '20230910/한라 4H/물리적피해,장비피해,잡초,광엽,피막이_1',
  'file_cnt': 501,
  'labeled_file_cnt': 0},
 {'path': '20230910/한라 4H/물리적피해,장비피해,잡초,광엽,피막이_2',
  'file_cnt': 501,
  'labeled_file_cnt'